# A minimal version of asprin

The goal of this project is to implement a minimal version of the system *asprin*.

Before you start, we recommend you to read sections 4 and 5 of the paper [1], and 
do the following little project on multi-shot solving:
* [../multi-shot-shell/multi-shot-shell.ipynb](../multi-shot-shell/multi-shot-shell.ipynb)

We introduce *asprin* below, but if you are interested, you can find more information at:
* the paper [2]
* the repository https://github.com/potassco/asprin
* these [slides](https://github.com/potassco/asprin/releases/download/v1/asprin-slides-btw17.pdf)
* the [Potassco Guide](https://github.com/potassco/guide/releases/download/v2.2.0/guide.pdf)


[1] [Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2023). How to Build Your Own ASP-based System?! Theory Pract. Log. Program., 23(1), 299–361.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/tplp/KaminskiRSW23.pdf)

[2] [Brewka, G., Delgrande, J. P., Romero, J., & Schaub, T. (2023). A general framework for preferences in answer set programming. Artif. Intell., 325, 104023.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/ai/BrewkaDRS23.pdf) 

# Clingo with optimization statements 

In *clingo* we can write logic programs with optimization statements and compute their optimal answer sets.

The following *clingo* program:

In [52]:
%%file example1.lp
dom(1..2).
1 { a(X) : dom(X) }.
#show a/1.

Overwriting example1.lp


has 3 answer sets: `{a(1)}`, `{a(2)}` and `{a(1),a(2)}`:

In [66]:
! clingo example1.lp 0 -V0

a(2)
a(1)
a(1) a(2)
SATISFIABLE


If we add a minimize statement:

In [54]:
%%file minimize.lp
#minimize{ 1,X : a(X) }.

Overwriting minimize.lp


then the resulting program has 2 **optimal** answer sets: `{a(1)}` and `{a(2)}`:

In [55]:
! clingo example1.lp minimize.lp --opt-mode=optN 0 -V0 --quiet=1

a(2)
Optimization: 1
a(1)
Optimization: 1
OPTIMUM FOUND


Here, option `--quiet=1` tells clingo to print only the optimal answer sets.

#### Formalities

A *clingo* program with minimize statements can be seen as a pair of:
* a logic program $P$
* a minimize statement $M$

Let $\mathcal{A}$ be the set of (ground) atoms occurring in the ground instantiation of $P$.

The minimize statement $M$ declares an irreflexive and transitive relation $\succ_M$ over the subsets of $\mathcal{A}$
(also called a [strict partial order](https://en.wikipedia.org/wiki/Partially_ordered_set#strict_partial_order)) where
* $X \succ_M Y$ if the weight of $X$ according to $M$ is less than the weight of $Y$ according to $M$

If $X \succ_M Y$ we say that $X$ is *better* than $Y$ wrt $M$.
  
Then, a set of atoms $X$ is an optimal answer set of $P$ and $M$ if:
* $X$ is an answer set of $P$, and
* there is no answer set $Y$ of $P$ such that $Y \succ_M X$

In our example:
* the set $\mathcal{A}$ is `{a(1),a(2)}`,
* the logic program $P$ has three answer sets `{a(1)}`, `{a(2)}` and `{a(1),a(2)}`,
* the minimize statement declares the strict partial order where:
  - `{}`$\succ_M$ `{a(1)}`, `{}`$\succ_M$ `{a(2)}`, `{}`$\succ_M$ `{a(1),a(2)}`,
  - `{a(1)}`$\succ_M$ `{a(1),a(2)}`, and
  - `{a(2)}`$\succ_M$ `{a(1),a(2)}`;
* the answer set `{a(1),a(2)}` is not optimal because both `{a(1)}` and `{a(2)}` are answer sets better than `{a(1),a(2)}` wrt $M$,
* while `{a(1)}` and `{a(2)}` are optimal answer sets because there is no answer set better than them (since `{}` is not an answer set).

# The ASP system *asprin*

The system *asprin* extends *clingo* by preference specifications, 
allowing more general optimization statements: subset preferences, conditional prefereces, their combination, and much more...

Comparing both systems:
* in *clingo* we can write logic programs with *optimization statements* and compute their optimal answer sets, while
* in *asprin* we can write logic programs with *preference specifications* and compute their optimal answer sets.

A nice feature of *asprin* is that we can define new types of preferences simply writing a logic program.

#### Formalities

An *asprin* program can be seen as a pair of:
* a logic program $P$
* a preference specification $S$

where now $S$ declares the irreflexive and transitive relation $\succ_S$ over the subsets of $\mathcal{A}$.

Optimal answer sets are defined just as above. 

A set of atoms $X$ is an optimal answer set of $P$ and $S$ if:
* $X$ is an answer set of $P$, and
* there is no answer set $Y$ of $P$ such that $Y \succ_S X$

### The same example in *asprin*

In asprin, instead of the previous minimize statement, we can write this *preference specification*:

In [28]:
%%file preference1.lp
#preference(p1,less(weight)) { 1,X :: a(X) : dom(X) }.
#optimize(p1).

Overwriting preference1.lp


The **first line** says that the preference statement `p1` is of type `less(weight)` and has the preference element `1,X :: a(X) : dom(X)`.

This statement declares the strict partial order $\succ_{\texttt{p1}}$, 
that is the same as the order declared by the minimize statement $M$ from above, i.e., $\succ_{\texttt{p1}} = \succ_M$.

The **second line** tells *asprin* to compute optimal models wrt `p1`.

To use *asprin* in this notebook, we clone it from its repository:

In [ ]:
! git clone https://github.com/potassco/asprin.git

Normally, we would install it with:
* `conda install -c potassco asprin --yes`

but today (23.5.2024) the latest version available via conda is not compatible with the latest version of *clingo* used in this notebook.

In [63]:
! python asprin/asprin/asprin.py example1.lp preference1.lp 0

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(2)
OPTIMUM FOUND
Answer: 2
a(1)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 2
Calls        : 7
Time         : 0.080s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.076s


Observe how we obtain the same optimal answer sets as above with the minimize statement.

## Extending the preference specification

Consider now this preference specification:

In [38]:
%%file preference2.lp
#preference(p1,less(weight)) { 1,X :: a(X) : dom(X) }.
#preference(p2,      subset) {        a(X) : dom(X) }.

#const s=p2.
#optimize(s).

Overwriting preference2.lp


The first line is the same as above.

The **second line** says that the preference statement `p2` is of type `subset` and has the preference element `a(X) : dom(X)`.

This declares the strict partial order $\succ_\texttt{p2}$ where $X \succ_\texttt{p2} Y$ if 
`{a(1),a(2)}` $\cap \ X \subset$ `{a(1),a(2)}` $\cap \ Y$.

It turns out that $\succ_\texttt{p2}$ is the same as $\succ_\texttt{p1}$ and as $\succ_M$. You can check this by hand.

Hence, the optimal models wrt `p2` are the same as before. 

The **last lines** tell *asprin* to compute those optimal models.

In [40]:
! python asprin/asprin/asprin.py example1.lp preference2.lp 0

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(2)
OPTIMUM FOUND
Answer: 2
a(1)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 2
Calls        : 7
Time         : 0.075s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.074s


We obtain the same answer sets if we set the constant `s` to be `p1` (adding `-c s=p1` to the previous call).

We can now extend the base program with these rules:

In [41]:
%%file example2.lp
dom(3).
a(3) :- a(2).
a(2) :- a(3).

Overwriting example2.lp


and we obtain 3 answer sets: 

In [42]:
! clingo example1.lp example2.lp 0 -V0

a(1)
a(3) a(2)
a(3) a(2) a(1)
SATISFIABLE


Only the first one is minimal wrt the minimize statement or wrt `p1`:

In [43]:
! clingo example1.lp example2.lp minimize.lp --opt-mode=optN 0 -V0 --quiet=1

a(1)
Optimization: 1
OPTIMUM FOUND


In [44]:
! python asprin/asprin/asprin.py example1.lp example2.lp preference2.lp 0 -c s=p1

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(1)
OPTIMUM FOUND

Models       : 1
  Optimum    : yes
  Optimal    : 1
Calls        : 4
Time         : 0.070s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.069s


but both `{a(1)}` and `{a(2),a(3)}` are optimal wrt `p2`:

In [45]:
! python asprin/asprin/asprin.py example1.lp example2.lp preference2.lp 0

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(1)
OPTIMUM FOUND
Answer: 2
a(3) a(2)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 2
Calls        : 7
Time         : 0.073s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.073s


How can this be the case?

The reason is that the order declared by `p2` is different from the order declared by either `p1` or $M$.

Now we have that $X \succ_\texttt{p2} Y$ if 
`{a(1),a(2),a(3)}` $\cap \ X \subset$ `{a(1),a(2),a(3)}` $\cap \ Y$.

But then, `{a(1)}` $\not\succ_\texttt{p2}$ `{a(2),a(3)}`. 
This makes sense, since `{a(1)}` is not a subset of `{a(2),a(3)}`.

On the other hand, `{a(1)}` $\succ_\texttt{p1}$ `{a(2),a(3)}`.
This makes sense, since the weight of `{a(1)}` is `1`, and this is less than the weight of `{a(2),a(3)}`, that is `2`.


Then, `{a(2),a(3)}` is optimal wrt `p2` because none of its subsets (`{}`, `{a(2)}` or `{a(3)}`) is an answer set, 
but it is not optimal wrt `p1` because `{a(1)}` is an answer set better than it (wrt `p1`).

## Preference types in *asprin*

We have seen the preference types `less(weight)` and `subset`. 

In *asprin*, defining a preference type is as simple as writing a logic program.

Here, we just give one example of such a program, 
but we do not explain it in detail, 
so do not worry if you do not understand it completely: 
it is not possible with the information that we are giving you!

If you want to know more, please have a look at our paper [2].

We can define the preference type `subset` with this *preference* program:

In [2]:
%%file mini-subset.lp
#program preference(subset).
better(P) :- preference(P,subset);
             not holds(X), holds'(X), preference(P,_,_,for(X),_);
             not holds(X) : preference(P,_,_,for(X),_), not holds'(X).
:- optimize(P), not better(P).

Overwriting mini-subset.lp


The first rule defines when:
* an answer set $A$ represented by facts of predicate `holds`
* is `better` than
* an answer set $B$ represented by facts of predicate `holds'`
* wrt a preference statement `P` of type `subset` represented by predicates `preference/2` and `preference/5`.

The integrity constraint enforces that $A$ is better than $B$ wrt the statement that is optimized.

The system *asprin* provides the atoms of predicates `preference/2`, `preference/5`, `optimize/1`, `holds/1` and `holds'/2`. The first two represent preference statements, the third one represents optimize statements, and 
the last two represent the two answer sets $A$ and $B$ that are being compared.


The preference program `mini-subset.lp`, given the facts about `p2`, defines the same order $\succ_\texttt{p2}$ that we specified above.

It is important to note that the core of *asprin* does not know anything about specific preference types.

Many of them, like `less(weight)` and `subset`, are already defined in [*asprin*'s library](https://github.com/potassco/asprin/blob/master/asprin/asprin_lib.lp),
that is loaded by default.

But we can extend the library, or even disable it and use our own preference programs, as in the next example 
where we use our `mini-subset.lp` instead of the library:

In [1]:
! python asprin/asprin/asprin.py example1.lp example2.lp preference2.lp 0 --no-asprin-lib mini-subset.lp --no-check

asprin version 3.1.2beta
Reading from example1.lp ...
Solving...
Answer: 1
a(1)
OPTIMUM FOUND
Answer: 2
a(3) a(2)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 2
Calls        : 7
Time         : 0.015s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.013s


Without the option `--no-check`, *asprin* complains because we have added no preference program for `less(weight)`, that shows up in `preference2.lp`.

# Task


The task of this project is to implement a minimal version of *asprin*, called *mini-asprin*.

The implementation of *asprin* has to parse the input files in order to:
* translate preference specifications to facts, and
* modify preference programs to avoid conflicts between programs.

To avoid these translations, in *mini-asprin*:
* the preference specification is given by *clingo* minimize statements, and
* we are given preference programs that do not have to be modified.

The input to *mini-asprin* is:
- some *clingo* program including minimize statements, and
- some *preference* program re-interpreting those minimize statements (see below).

Given that input, 
*mini-asprin* should print the answer sets of the *clingo* program 
that are optimal wrt the re-interpretation of the minimize statements. 

## An example

If:

* the input consists of the files `example1.lp example2.lp minimize.lp`, and
* the preference program (in `less-weight.lp`) interprets minimize statements as usual,

then we should obtain the unique optimal answer set `{a(1)}`.

In the next cell,
we used our final version of `mini-asprin.py` to generate the output:

In [9]:
! clingo example1.lp example2.lp minimize.lp --output=reify | python mini-asprin.py - less-weight.lp 0 --sign-def=pos

mini-asprin version 1.0
Reading from - ...
Solving...
Answer: 1
a(2) a(1) a(3)
Solving...
Answer: 1
a(2) a(3)
Solving...
Answer: 1
a(1)
Solving...
OPTIMUM FOUND
Solving...
UNSATISFIABLE

Models       : 3
Calls        : 5
Time         : 0.003s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.003s


We use option `--sign-def` just for presentation purposes. 
This option tells *clingo* to make atoms true by default, leading it first to the answer set `{a(1),a(2),a(3)}`. 
Without option `--sign-def`, our version computes directly the optimal model `{a(1)}` and does not go through any non-optimal models.

In the example, after finding `{a(1),a(2),a(3)}`, *mini-asprin*:
* looks for an answer set better than `{a(1),a(2),a(3)}` and finds `{a(2),a(3)}`, 
* then it looks for an answer set better than `{a(2),a(3)}` and finds `{a(1)}`, 
* then it looks for an answer set better than `{a(1)}`,
and since it finds no one,
it knows that `{a(1)}` is optimal and prints the message `OPTIMUM FOUND`.

Finally, in the last `Solving...` call, *mini-asprin* looks for some answer set that is neither worse nor the  same as `{a(1)}`,
and since it does not find any, it finishes
(and *clingo* prints `UNSATISFIABLE`, because this is the result of the last `solve()` call).

If the preference program (in `subset.lp`, see below) interprets the minimize statement as a subset preference, 
then we should obtain the two optimal answer sets `{a(1)}` and `{a(2), a(3)}`.

Again, we used our final version of `mini-asprin.py` to generate the output, together with option `--sign-def=pos`:

In [10]:
! clingo example1.lp example2.lp minimize.lp --output=reify | python mini-asprin.py - subset.lp 0 --sign-def=pos

mini-asprin version 1.0
Reading from - ...
Solving...
Answer: 1
a(2) a(1) a(3)
Solving...
Answer: 1
a(2) a(3)
Solving...
OPTIMUM FOUND
Solving...
Answer: 1
a(1)
Solving...
OPTIMUM FOUND
Solving...
UNSATISFIABLE

Models       : 3
Calls        : 6
Time         : 0.004s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.004s


In this case:
* there is no answer set better than `{a(2),a(3)}`, so *mini-asprin* prints `OPTIMUM FOUND`,
* then it looks for an answer set that is neither worse nor the same as `{a(2),a(3)}` and finds `{a(1)}`,
* then it looks for an answer set better than `{a(1)}`, and since it finds no one, it prints `OPTIMUM FOUND`,
* finally it looks for an answer set that is neither worse nor the same as `{a(2),a(3)}` and `{a(1)}`, and since it finds no one, it finishes.

## Approach

We recommend you to extend the initial script that comes with this notebook. As you can see, it uses meta-programming.

In its current form, the script loads the input and the meta-encoding, grounds both together and solves.

It computes normal answer sets of the input program.

In [28]:
! cat mini-asprin.py

import sys
from clingo.application import clingo_main, Application
from clingo.script import enable_python
from clingo.symbol import Number, Function

enable_python()

META = """
conjunction(B) :- literal_tuple(B),
        hold(L,0) : literal_tuple(B, L), L > 0;
    not hold(L,0) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L,0), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L,0), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A,0) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A,0) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).
"""

HOLD = """
hold(A,m) :- A = @get_hold().
"""

DELETE = """
:- hold(A,0) : hold(A,m);
   hold(A,m) : hold(A,0).
"""

class MiniAsprinApp(Application):

    program_name = "mini-asprin"
    version = "1.0"
    
    def main(self, ctl, files):
        for path 

The script uses a modification of the common meta-encoding,
where true atoms are represented by `hold(A,0)` instead of `hold(A)`.

The final *mini-asprin* will often compute many answer sets.
We can:
* assign a number greater than `0` to each of them: `1`, `2`, `...`, and
* represent the true atoms in answer set `M` by `hold(A,M)`.

We can use the program `HOLD` to add those `hold(A,M)` atoms after computing answer set `M`.

Program `DELETE` can be used to eliminate the answer set `m` 
from our program, where `m` should be the number of an answer set.

Let's have a look at `subset.lp`:

In [68]:
! cat subset.lp

#program preference(m1,m2).
better(m1,m2) :- minimize(P,B), P = 0,
  1 { not hold(L,m1) :     hold(L,m2), weighted_literal_tuple(B, L,W), L > 0 ;
          hold(L,m1) : not hold(L,m2), weighted_literal_tuple(B,-L,W), L > 0 },
      not hold(L,m1) : not hold(L,m2), weighted_literal_tuple(B, L,W), L > 0 ;
          hold(L,m1) :     hold(L,m2), weighted_literal_tuple(B,-L,W), L > 0 .


The rule defines when:
* an answer set `m1` represented by facts of the form `holds(L,m1)`
* is `better` than
* an answer set `m2` represented by facts of the form `holds(L,m2)`
* wrt a minimize statement of priority `0`:
  - represented by predicates `minimize/2` and `weighted_literal_tuple/3`, and
  - interpreted as a subset preference over the literals occurring in the statement.
 
The rule derives `better` whenever:
* some literal is not satisfied by `m1` but is satisfied by `m2` (this is represented by the cardinality constraint), and
* `m1` does not satisfy the literals that are not satisfied by `m2` (this is represented by the last two conditional literals).

Observe that this preference program ignores the weights.

## Computing one optimal answer set

The idea of the method is as follows:
* first we compute some answer set `1`,
* then we look for some answer set `2` better than `1`;
* if there is no such answer set, then `1` is optimal,
* otherwise we look for some answer set `3` better than `2`;
* if there is no such answer set, then `2` is optimal,
* otherwise we continue...

How do we implement this idea? For example, as follows.

The first call to `clt.solve()` gives us some answer set `X`.

We take the set of atoms `A` such that `hold(A,0)` is in `X` and give this set the number `1`.

Note that this set `1` is an answer of the input logic program.

Next, we:
* add the facts `hold(A,1)` for all atoms in the answer set `1`,
* add the program `preference(m1,m2)` with `m1=0` and `m2=1` so that `better(0,1)`
  tells us if answer set `0` is better than `1`,
* enforce that `better(0,1)` holds.

This guarantees that the next answer set (if any) is better than `1`.

Then, we call `ctl.solve()`.

If the result is `UNSAT`, then we know that there is no better answer set than `1`.

Hence, `1` is an optimal answer set and we print `OPTIMUM FOUND`.

If the result is not `UNSAT`, then we obtain some answer set `X`.

As before, we take the set of atoms `A` such that `hold(A,0)` is in `X`, 
and this time we give this set the number `2`.

We know that the set `2` is better than `1`.

Next, we:
* add the facts `hold(A,2)` for all atoms in the answer set `2`,
* add the program `preference(m1,m2)` with `m1=0` and `m2=2` so that `better(0,2)`
  tells us if answer set `0` (the one computed at each `solve` call) is better than `2`,
* enforce that `better(0,2)` holds.

This guarantees that the next answer set (if any) is better than `2`.

Then, we call `ctl.solve()`.

If the result is `UNSAT`, then `2` is an optimal answer set and we print `OPTIMUM FOUND`.

If the result is not `UNSAT`, then we continue with this process...

... until we obtain `UNSAT`, 
in which case we know that the last computed answer set contains an optimal answer.

## Computing all answer sets

To compute all answer sets, after we compute one optimal model, 
we extend our program to eliminate:
* the answer sets that are worse than the last (optimal) one, and
* the answer sets that are the same as the last (optimal) one.

We also have to stop enforcing that the next answer set is better than any other.

In this way, the next solve call will give us some answer set (if any)
that is neither worse nor the same as the optimal ones that we have already computed.

To do this in the implementation, after we have computed one optimal answer set with number `n`, we can:
* add the program `preference(m1,m2)` with `m1=n` and `m2=0` so that `better(n,0)`
  tells us if answer set `n` is better than `0`,
* enforce that `better(n,0)` does not hold,
* delete answer set `n` using program `DELETE`, and
* stop enforcing any atom of the form `better(0,...)`

Then, we call `ctl.solve()`.

If the result is `UNSAT`, then we know that there are no more optimal answer sets.

If the result is not `UNSAT`, then we obtain some answer set `X` 
whose `hold(A,0)` atoms represent an answer set `n+1` 
such that:
* `n+1` is not worse than the previous optimal answer sets, and
* `n+1` is different than the previous optimal answer sets.

Once the answer `n+1` is computed, we just have to run the previous method to compute one optimal answer set, and continue from there...

## Checking your mini-asprin

You can check your script using the following encoding:

In [6]:
%%file test.lp
#const n=1.
v(X) :- X = 1..3*n, X \ 3 = 1.
1 { a(X); a(X+1); a(X+2) } :- v(X). 
            a(X+1) :- a(X+2), v(X). 
            a(X+2) :- a(X+1), v(X).

Overwriting test.lp


This generalizes our `example1.lp` and `example2.lp`. 

For `n=1`, we obtain the same answer sets as putting both files together.

If the minimize statement at `minimize.lp` is interpreted as `less(weight)`, 
then for any value of `n` there is a unique optimal answer set.

You can use the line below to check this for `n=1..5`. It should print five `1`'s.

In [27]:
! for n in {1..5}; do clingo test.lp minimize.lp --output=reify -c n=$n | python mini-asprin.py - less-weight.lp 0 > out.txt; grep --count "OPTIMUM FOUND" out.txt; done

1
1
1
1
1


If the minimize statement at `minimize.lp` is interpreted as `subset`, 
then for any value of `n` there are `2^n` optimal answer sets.

Accordingly, the cell below should print the numbers `2`, `4`, `8`, `16` and `32`.

In [26]:
! for n in {1..5}; do clingo test.lp minimize.lp --output=reify -c n=$n | python mini-asprin.py - subset.lp 0 > out.txt; grep --count "OPTIMUM FOUND" out.txt; done

2
4
8
16
32


## Hints

* You can use option `--output-debug=[text,translate,all]` to print the rules grounded by *clingo*

## Extensions

* You can extend the files `less-weight.lp` and `subset.lp` to handle priority levels other than `0`.
  You can aggregate those levels using a lexicographic order (as in *clingo* minimize statements) or using a pareto order.
* You can add an option to project the solutions over the atoms occurring in the minimize statements.
* You can add an option for computing optimal models using preference approximations and domain-specific heuristics.
* You can add an option for computing optimal models using preference approximations and clingo minimize statements.
* You can add an option to solve queries over optimal models.